## Access to the WIND API and get the stocklist and features
Do not need to run again.

In [7]:
import datetime
import numpy as np
import pandas as pd

In [9]:
#access to the API of WIND
from WindPy import * 
w.start()
enddate="2018-04-30"
startdate=datetime.datetime(2010, 1, 1, 0, 0) #only get the stock IPO after 2010

In [ ]:
#get all A-shares on the market
allstock=w.wset("sectorconstituent","date="+enddate+";sectorid=a001010100000000;field=wind_code")
allstockcode=allstock.Data[0]
ipodate=w.wss(allstockcode, "ipo_date")
ipodate=ipodate.Data[0]
finddate=np.nonzero([x>startdate for x in ipodate])[0]
allstock=np.array(allstockcode)[finddate]

In [ ]:
#get the features we need 
IPOdata=w.wss(list(allstock), "industry_gics,mkt,backdoor,ipo_date,ipo_price,ipo_amount,ipo_expense,\
ipo_puboffrdate,ipo_leadundr,ipo_type,ipo_npctchange,ipo_pctchange,ipo_turn,ipo_nturn","unit=1;industryType=1;n=10")
alldata=pd.DataFrame(IPOdata.Data).T
alldata.columns=IPOdata.Fields
alldata.index=allstock
alldata=alldata.dropna()#1828*14

In [ ]:
#save the data as csv
path='C:\\Users\\admin\\Documents\\GitHub\\2017.M3.TQF-Forcasting-price-change-on-IPO-day\\'
alldata.to_csv(path+'alldata.csv',header=True,index=True)

In [14]:
#path='C:\\Users\\admin\\Documents\\GitHub\\2017.M3.TQF-Forcasting-price-change-on-IPO-day\\'
alldata=pd.read_csv(path+'alldata.csv',encoding='gbk',index_col=0)
alldata.head()

,INDUSTRY_GICS,MKT,BACKDOOR,IPO_DATE,IPO_PRICE,IPO_AMOUNT,IPO_EXPENSE,IPO_PUBOFFRDATE,IPO_LEADUNDR,IPO_TYPE,IPO_NPCTCHANGE,IPO_PCTCHANGE,IPO_TURN,IPO_NTURN
002329.SZ,日常消费,中小企业板,否,2010/1/6 0:00,20.10,27000000,27813988.42,2009/12/15 0:00,光大证券股份有限公司,"网下询价,上网定价",6.780250,43.084577,72.658616,3.794127
002330.SZ,日常消费,中小企业板,否,2010/1/6 0:00,13.18,63000000,31017850.00,2009/12/15 0:00,民生证券股份有限公司,"网下询价,上网定价",-9.216013,81.942337,82.889379,3.078005
002331.SZ,信息技术,中小企业板,否,2010/1/6 0:00,27.00,14000000,33497830.00,2009/12/15 0:00,国元证券股份有限公司,"网下询价,上网定价",26.786953,60.111111,74.613473,4.006118
002332.SZ,医疗保健,中小企业板,否,2010/1/12 0:00,8.20,85400000,44224230.59,2009/12/18 0:00,中信建投证券股份有限公司,"网下询价,上网定价",-19.221168,144.268293,86.137165,2.500126
002333.SZ,工业,中小企业板,否,2010/1/12 0:00,22.00,39200000,28778000.00,2009/12/18 0:00,德邦证券股份有限公司,"网下询价,上网定价",-11.450142,44.500000,73.111033,2.442266


In [12]:
[M,N]=alldata.shape
ipodata=alldata.copy()
#calculate the how many days were spend on ipo,make it a new variable
ipodate=[datetime.datetime.strptime(alldata['IPO_DATE'][x],'%Y-%m-%d %H:%M:%S') for x in range(M)]
pubofferdate=[datetime.datetime.strptime(alldata['IPO_PUBOFFRDATE'][x],'%Y-%m-%d %H:%M:%S') for x in range(M)]
ipotime=[(ipodate[x]-pubofferdate[x]).days for x in range(M)]
ipodata['ipotime']=ipotime

#transfer the date of ipo to year
ipodateYEAR=[ipodate[x].year for x in range(M)]
ipodata['IPO_DATE']=ipodateYEAR

In [ ]:
#for those before 2014,use the first day return
#for those after 2014,use the first 10-days return
ipodata['iporeturn']=ipodata['IPO_PCTCHANGE']
ipodata['iporeturn'][ipodata['IPO_DATE']>2013]=ipodata['IPO_NPCTCHANGE'][ipodata['IPO_DATE']>2013]
ipodata['ipoturn']=ipodata['IPO_TURN']
ipodata['ipoturn'][ipodata['IPO_DATE']>2013]=ipodata['IPO_NTURN'][ipodata['IPO_DATE']>2013]
ipodata=ipodata.drop(ipodata.columns[10:14],1)
ipodata=ipodata.drop(ipodata.columns[8],1)

ipodata.to_csv(path+'ipodata.csv',header=True,index=True)

In [15]:
ipodata=pd.read_csv(path+'ipodata.csv',encoding='gbk',index_col=0)
ipodata.head()

,INDUSTRY_GICS,MKT,BACKDOOR,IPO_DATE,IPO_PRICE,IPO_AMOUNT,IPO_EXPENSE,IPO_LEADUNDR,IPO_TYPE,ipotime,iporeturn,ipoturn
002329.SZ,日常消费,中小企业板,否,2010,20.10,27000000,27813988.42,光大证券股份有限公司,"网下询价,上网定价",22,43.084577,72.658616
002330.SZ,日常消费,中小企业板,否,2010,13.18,63000000,31017850.00,民生证券股份有限公司,"网下询价,上网定价",22,81.942337,82.889379
002331.SZ,信息技术,中小企业板,否,2010,27.00,14000000,33497830.00,国元证券股份有限公司,"网下询价,上网定价",22,60.111111,74.613473
002332.SZ,医疗保健,中小企业板,否,2010,8.20,85400000,44224230.59,中信建投证券股份有限公司,"网下询价,上网定价",25,144.268293,86.137165
002333.SZ,工业,中小企业板,否,2010,22.00,39200000,28778000.00,德邦证券股份有限公司,"网下询价,上网定价",25,44.500000,73.111033
